In [15]:
import pydriller
import pandas as pd
from tqdm import tqdm


# load dataset
commit_df = pd.read_csv(
    "labelledCommitDataset/Commits.csv",
    names=["commit_id", "commit_hash", "project_id", "author_id", "committer_id", "date", "title"]
)
survey_df = pd.read_csv(
    "labelledCommitDataset/SurveyResults.csv",
    names= [
        "commit_id",
        "sw_adaptive",
        "sw_corrective",
        "sw_perfective",
        "nfr_maintainability",
        "nfr_usability",
        "nfr_functionality",
        "nfr_reliability",
        "nfr_efficiency",
        "nfr_portability",
        "nfr_none",
        "hl_forward",
        "hl_reengineering",
        "hl_corrective",
        "hl_management",
    ]
)

In [16]:
#commit_df = commit_df[commit_df['project_id'] == 1]
#num_commits = len(commit_df.index)
#survey_df = survey_df.head(num_commits)
survey_df["commit_hash"] = commit_df["commit_hash"]
survey_df["project_id"] = commit_df["project_id"]
#print(commit_df.head())
#print(survey_df.head())
print(len(survey_df.index))


967


In [25]:
project_ids = [1, 2, 4, 6]
select_project_df = survey_df[
    (survey_df["project_id"] == 1) |
    (survey_df["project_id"] == 2) |
#    (survey_df["project_id"] == 3) |
    (survey_df["project_id"] == 4) |
#    (survey_df["project_id"] == 5) |
    (survey_df["project_id"] == 6)
]
print(len(select_project_df))

768


In [3]:
commit_row = commit_df[commit_df["commit_hash"] == "37819b1b6409eb7cbaa96366bb91f6557f821b11"].iloc[0]
commit_id = commit_row["commit_id"]
print("commit_id {}".format(commit_id))
survey_row = survey_df[survey_df["commit_id"] == commit_id].iloc[0]
#print(survey_row)
classifications = []
for col_name, col_val in survey_row.iteritems():
    #print("{}: {}".format(col_name, column))
    if (col_val == 1):
        classifications.append(col_name)
print(classifications)

commit_id 10
['sw_adaptive', 'nfr_functionality', 'hl_forward']


In [33]:
from gitProfileSet import commitType

# Load git repo
repos = [
#    "gitRepos/valadoc",
#    "gitRepos/vala",
#    "gitRepos/drupal",
#    "gitRepos/tapiji",
#    "gitRepos/org.eclipse.mylyn",
    "gitRepos/deltaspike",
]
#project_id = 1
# mine commits
#langList =["cpp", "c"]
classifications = []
for repo in repos:
    miner = pydriller.repository_mining.RepositoryMining(
        repo,
        only_no_merge=True
    )
    repository = pydriller.GitRepository(repo)
    print("Scanning repo: " + miner._path_to_repo)

    print("num Commits: {}".format(len(list(miner.traverse_commits()))))
    # repo is path to git repository


    for commit in miner.traverse_commits():
        commitLabel = commitType.categorize(commit)
        #print("Added: {}".format(added))
        #print("{}: {}".format(commit.hash, commitLabel))
        classifications.append((commit.hash, commitLabel.value))
print(len(classifications))

Scanning repo: gitRepos/deltaspike
num Commits: 2293
2293


In [34]:
classification_dict = {
    'FC': 'hl_forward',
    'BF': 'hl_corrective',
    'RF': 'hl_reengineering',
    'OTH': 'hl_management',
}
labelled_df = pd.DataFrame(classifications, columns=['commit_hash', 'label'])
#print(labelled_df.head())
#print(survey_df.head())
classified_df = pd.merge(survey_df, labelled_df, on=['commit_hash'])
#print(classified_df)
print(len(classified_df.index))

134


In [11]:
# Get number of matches
num_forward = len(classified_df[
    (classified_df['hl_forward'] == 1) &
    (classified_df['label'] == 'FC')]
)
num_bugfix = len(classified_df[
    (classified_df['hl_forward'] == 0) &
    (classified_df['hl_corrective'] == 1) &
    (classified_df['label'] == 'BF')]
)
num_refactor = len(classified_df[
    (classified_df['hl_forward'] == 0) &
    (classified_df['hl_corrective'] == 0) &
    ((classified_df['hl_reengineering'] == 1) |
    (classified_df['hl_management'] == 1)) &
    (classified_df['label'] == 'RF')]
)
num_other = len(classified_df[
    (classified_df['hl_forward'] == 0) &
    (classified_df['hl_corrective'] == 0) &
    (classified_df['hl_reengineering'] == 0) &
    (classified_df['hl_management'] == 0) &
    (classified_df['label'] == 'OTH')]
)
print("num_forward: {}".format(num_forward))
print("num_bugfix: {}".format(num_bugfix))
print("num_refactor: {}".format(num_refactor))
print("num_other: {}".format(num_other))

total_correct = num_forward + num_bugfix + num_refactor + num_other
total_rows = len(classified_df.index)
print("accuracy: {}".format(total_correct / total_rows))

num_forward: 11
num_bugfix: 4
num_refactor: 15
num_other: 0
accuracy: 0.25862068965517243


In [21]:
classified_df[classified_df['label'] == 'OTH']

,commit_id,sw_adaptive,sw_corrective,sw_perfective,nfr_maintainability,nfr_usability,nfr_functionality,nfr_reliability,nfr_efficiency,nfr_portability,nfr_none,hl_forward,hl_reengineering,hl_corrective,hl_management,commit_hash,label
0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0e166c04eb66137bc2b8ed69388f3fc620298e4d,OTH
2,3,1,0,0,0,1,1,0,0,0,0,1,0,0,0,300e643ce781132b7276962b1a8223b3ebde5fde,OTH
3,4,1,0,0,0,0,1,0,0,0,0,1,0,0,0,eee27a89faa533249f7b3aad7af23c0fd61dfc6e,OTH
4,5,0,1,0,0,0,1,0,0,0,0,0,0,1,0,afb7c165c79a7ed483b1871c1fb4601ce07b4918,OTH
5,6,0,0,1,0,0,0,0,0,0,1,0,0,0,1,c0adeabd80460d704d832b12ddc8b694ce4e52ea,OTH
7,8,1,0,0,0,0,1,0,0,0,0,1,0,0,0,faeaecd49a7c17ec7631cec7a859933976ed7068,OTH
8,9,1,0,0,0,0,1,0,0,0,0,1,0,0,0,8c9507ec33019b88dd43ab47e1bd50d22332740b,OTH
9,10,1,0,0,0,0,1,0,0,0,0,1,0,0,0,37819b1b6409eb7cbaa96366bb91f6557f821b11,OTH
12,13,0,1,0,0,0,1,0,0,0,0,0,0,1,0,aa0fa2da0f5e0b1ab6ace926866342b803504941,OTH
13,14,0,1,0,0,0,1,1,0,0,0,0,0,1,0,89ecb6a91b5bfbe7d1fbf7703b09519079bd6a33,OTH
